# 3.4 다중 분류

## 3.4.0 이전 내용 중 필요한 로직

In [1]:
## 기본

import warnings
warnings.filterwarnings('ignore')

# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# MNIST 데이터셋 불러오기
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)

# 데이터셋 분리
X, y = mnist['data'], mnist['target']

# 샘플 데이터 (숫자 5)
some_digit = X[0]

# 레이블 데이터 타입 변환(문자열 -> 숫자)
y = y.astype(np.uint8)

# 테스트 세트 분리
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

print('done')

done


<br>

## 3.4.1 다중 분류기 (multiclass classifier)

- 이진 분류
  - 두 개의 클래스를 구별  
  
  
- 다중 분류기 (multiclass classifier)
  - 다항 분류기(multinomial classifier) 라고도 불림
  - 둘 이상의 클래스를 구별

<br>

## 3.4.2 다중 분류가 가능한 알고리즘

- SGD 분류기
- 랜덤 포레스트 분류기
- 나이브 베이즈(naive bayes) 분류기

**cf) 이진 분류만 가능한 알고리즘**

- 로지스틱 회귀
- 서포트 벡터 머신 분류

<br>

## 3.4.3 이진 분류기 여러 개를 사용한 다중 클래스 분류 전략

- 특정 숫자 하나만 구분하는 숫자별 이진 분류기 10개(0~9)를 훈련 시켜 클래스가 10개인 숫자 이미지 분류 시스템을 만들 수 있음

**1) OvR(one-versus-the-rest) 전략**

- 이미지 분류 시 각 분류기의 결정 점수 중에서 가장 높은 것을 클래스로 선택하는 전략
- OvA(one-versus-all) 전략이라고도 불림
- 대부분의 이진 분류 알고리즘에서는 해당 전략을 선호

**2) OvO(one-versus-one) 전략**

- 0과 1 구별, 0과 2 구별, 1과 2 구별 등과 같이 각 숫자의 조합마다 이진 분류기를 훈련시키는 전략
- 클래스가 $N$개라면 분류기는 $N \times \left( N-1 \right) / 2$ 개가 필요
  - $n$개의 원소에서 $k$를 뽑을 수 있는 조합의 수 (이항 계수 공식)
- ex) MNIST 문제에서는 45개의 분류기를 훈련시켜야 한다.
  - $10 \times 9 \; / \; 2 = 45$
  - 이미지 하나를 분류하려면 45개 분류기 모두를 통과시켜서 가장 많이 양성으로 분류된 클래스 선택
- 주요 장점
  - 각 분류기의 훈련에 전체 훈련 세트 중 구별할 두 클래스에 해당하는 샘플만 필요
- 훈련 세트의 크기에 민감한 일부 알고리즘(ex. 서포트 벡터 머신)에서는 OvO 전략 선호
  - 큰 훈련 세트에서 몇 개의 분류기를 훈련시키는 것보다 작은 훈련 세트에서 많은 분류기를 훈련시키는 쪽이 빠르기 때문

- 다중 클래스 분류 작업에 이진 분류 알고리즘을 선택하면 사이킷런이 알고리즘에 따라 자동으로 OvR 또는 OvO를 실행한다.
  - `sklearn.svm.LinearSVC` : OvR 전략 사용
  - `sklearn.svm.SVC` : OvO 전략 사용

In [2]:
from sklearn.svm import SVC

svm_clf = SVC()
svm_clf.fit(X_train[:1000], y_train[:1000])
svm_clf.predict([some_digit]) # 5

array([5], dtype=uint8)

<br>

- 내부에서는 사이킷런이 OvO 전략을 사용해 10개의 이진 분류기를 훈련시키고 각각의 결정 점수를 얻어 점수가 가장 높은 클래스를 선택한다.

In [3]:
some_digit_scores = svm_clf.decision_function([some_digit])
some_digit_scores

array([[ 1.75828215,  2.74956232,  6.13809559,  8.2853702 , -0.28728967,
         9.30119996,  0.74228825,  3.79256174,  7.20847395,  4.85762716]])

In [4]:
np.argmax(some_digit_scores)

5

<br>

- 분류기가 훈련될 때 `classes_` 속성에 타깃 클래스의 리스트를 값으로 정렬하여 저장한다.
- `classes_` 배열에 있는 각 클래스의 인덱스가 클래스 값 자체와 같다.
  - i.e. 인덱스 5에 해당하는 클래스의 값은 5이다.
- 하지만 일반적으로 이런 경우는 드물다.

In [5]:
svm_clf.classes_

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [6]:
svm_clf.classes_[5]

5

<br>

## 3.4.4 OvO, OvR 전략 강제 적용

- 사이킷런에서 OvO나 OvR을 사용하도록 강제하려면 다음 2가지 클래스를 사용한다.
  - `OneVsOneClassifier` : OvO
  - `OneVsRestClassifier` : OvR
- 간단하게 이진 분류기 인스턴스를 만들어 객체를 생성할 때 전달하면 된다. (이진 분류기일 필요는 없음, 다중 분류기도 가능)

<br>

### 3.4.4.1 `SVC` 기반으로 OvR 전략을 사용하는 다중 분류기 구현

In [7]:
from sklearn.multiclass import OneVsRestClassifier

ovr_clf = OneVsRestClassifier(SVC())
ovr_clf.fit(X_train[:1000], y_train[:1000])

OneVsRestClassifier(estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                                  gamma='scale', kernel='rbf', max_iter=-1,
                                  probability=False, random_state=None,
                                  shrinking=True, tol=0.001, verbose=False),
                    n_jobs=None)

In [8]:
ovr_clf.predict([some_digit])

array([5], dtype=uint8)

In [9]:
len(ovr_clf.estimators_)

10

<br>

## 3.4.5 `SGDClassifier`(또는 `RandomForestClassifier`) 기반 다중 분류기 구현

- SGD 분류기는 직접 샘플을 다중 클래스로 분류할 수 있기 때문에 별도로 사이킷런의 OvR이나 OvO를 적용할 필요가 없다.

In [10]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3)
sgd_clf.fit(X_train[:1000], y_train[:1000])

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [11]:
sgd_clf.predict([some_digit])

array([5], dtype=uint8)

<br>

- `decision_function()` 메서드는 각 샘플에 대해 클래스마다 하나의 값을 반환한다.
- SGD 분류기가 클래스마다 분류한 점수를 확인해보자.

In [12]:
sgd_clf.decision_function([some_digit])

array([[-5729001.13389174, -4714442.8733707 , -3391334.3645554 ,
         -270280.23939202, -5048896.19628065,   478544.38444117,
        -6777765.71045801, -3674811.21587839, -2799574.15341868,
        -3214596.72388322]])

- 이 결과에서 분류기가 예측 결과에 강한 확신을 보이고 있음을 알 수 있다.
  - 대부분의 점수가 큰 음수이다.
  - 반면 클래스 5의 점수는 3343.9 이다.

<br>

## 3.4.5 다중 분류기 평가

- 분류기 평가에는 일반적으로 교차 검증을 사용한다.
- `cross_val_score()` 함수를 사용해 `SGDClassifier`의 정확도를 확인해보자.

In [13]:
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train[:1000], y_train[:1000], 
                cv=3, scoring='accuracy')

array([0.84131737, 0.81081081, 0.81081081])

- 모든 테스트 폴드에서 84% 이상을 얻었다.
- 랜덤 분류기를 사용했다면 10% 정확도를 얻었을 것이므로 이 점수가 아주 나쁘진 않지만 성능을 더 높일 여지가 있다.

<br>

## 3.4.6 다중 분류기 성능 향상

- ex) 입력 스케일을 조정하면 정확도를 89% 이상으로 높일 수 있다.

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
cross_val_score(sgd_clf, X_train_scaled[:1000], y_train[:1000], 
                cv=3, scoring='accuracy')

array([0.83233533, 0.81981982, 0.78978979])

<br>

## 3.4.7 KNN 분류기

### 3.4.7.1 KNN 분류기 학습

In [16]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=4)
knn_clf.fit(X_train[:1000], y_train[:1000])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='distance')

<br>

### 3.4.7.2 KNN 분류기 예측

In [17]:
y_knn_pred = knn_clf.predict(X_test[:1000])

<br>

### 3.4.7.3 KNN 분류기 평가

In [18]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test[:1000], y_knn_pred)

0.827